# Production dataset preparation

In [ ]:
# Skip this cell, at first
import os
os.chdir('../..')
os.getcwd()

In [1]:
import pandas as pd
import numpy as np
from CI_Experiments.config import PROJECT_DIR # if this line results ModuleNotFoundError, then execute the cell above

In [2]:
UNPREPARED_PATH = PROJECT_DIR / 'data/unprepared_process_logs'
PRODUCTION_DATA = UNPREPARED_PATH / 'Production.csv'
DESTINATION_DATA = PROJECT_DIR / 'data/prepared_process_logs/Production/activity/activity.csv'

In [3]:
unprepared_data = pd.read_csv(PRODUCTION_DATA, sep=';')
unprepared_data.head()

,Part_Desc_,Rework,Work_Order_Qty,Case ID,label,Activity,Resource,Report_Type,Resource.1,Qty_Completed,...,activity_duration,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases
0,Cable Head,missing,250,Case178,regular,Round Grinding - Machine 3,ID4445,D,Machine 3 - Round Grinding,31,...,290.0,2012-01-02 04:50:00,290,1,0,4,0.0,0.0,1,1
1,Electrical Contact,missing,320,Case238,deviant,Turning & Milling - Machine 4,ID4794,S,Machine 4 - Turning & Milling,0,...,360.0,2012-01-02 07:00:00,420,1,0,7,0.0,0.0,1,2
2,Cable Head,missing,200,Case187,deviant,Lapping - Machine 1,ID4882,D,Machine 1 - Lapping,0,...,150.0,2012-01-02 09:30:00,570,1,0,9,0.0,0.0,1,3
3,Cable Head,missing,250,Case178,regular,Round Grinding - Machine 2,ID4355,S,Machine 2 - Round Grinding,0,...,60.0,2012-01-02 11:00:00,660,1,0,11,370.0,370.0,2,4
4,Compound Dies,missing,18,Case263,regular,Turning & Milling - Machine 8,ID4219,S,Machine 8 - Turning & Milling,0,...,120.0,2012-01-02 11:00:00,660,1,0,11,0.0,0.0,1,4


In [4]:
unprepared_data.columns

Index(['Part_Desc_', 'Rework', 'Work_Order_Qty', 'Case ID', 'label',
       'Activity', 'Resource', 'Report_Type', 'Resource.1', 'Qty_Completed',
       'Qty_for_MRB', 'activity_duration', 'Complete Timestamp',
       'timesincemidnight', 'month', 'weekday', 'hour', 'timesincelastevent',
       'timesincecasestart', 'event_nr', 'open_cases'],
      dtype='object')

In [5]:
four_columns_for_activity = [
    'Case ID',
    'label',
    'Activity',
    'Complete Timestamp'
]

In [6]:
activity_data = unprepared_data[four_columns_for_activity]
activity_data.head()

,Case ID,label,Activity,Complete Timestamp
0,Case178,regular,Round Grinding - Machine 3,2012-01-02 04:50:00
1,Case238,deviant,Turning & Milling - Machine 4,2012-01-02 07:00:00
2,Case187,deviant,Lapping - Machine 1,2012-01-02 09:30:00
3,Case178,regular,Round Grinding - Machine 2,2012-01-02 11:00:00
4,Case263,regular,Turning & Milling - Machine 8,2012-01-02 11:00:00


In [7]:
activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)
activity_data.head()

/var/folders/2v/0cy8xvx12vl1y6x611k263_w0000gn/T/ipykernel_84225/2959549615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)


,Case ID,label,Activity,Complete Timestamp
0,Case178,0,Round Grinding - Machine 3,2012-01-02 04:50:00
1,Case238,1,Turning & Milling - Machine 4,2012-01-02 07:00:00
2,Case187,1,Lapping - Machine 1,2012-01-02 09:30:00
3,Case178,0,Round Grinding - Machine 2,2012-01-02 11:00:00
4,Case263,0,Turning & Milling - Machine 8,2012-01-02 11:00:00


In [8]:
activity_data = activity_data.rename(columns={'label': 'Outcome', 'Complete Timestamp': 'Timestamp'})
activity_data.head()

,Case ID,Outcome,Activity,Timestamp
0,Case178,0,Round Grinding - Machine 3,2012-01-02 04:50:00
1,Case238,1,Turning & Milling - Machine 4,2012-01-02 07:00:00
2,Case187,1,Lapping - Machine 1,2012-01-02 09:30:00
3,Case178,0,Round Grinding - Machine 2,2012-01-02 11:00:00
4,Case263,0,Turning & Milling - Machine 8,2012-01-02 11:00:00


In [10]:
activity_data.to_csv(DESTINATION_DATA, index=False)

In [9]:
len(activity_data['Activity'].unique())

26

In [11]:
np.mean(activity_data.groupby('Case ID').count()['Activity'])

11.313636363636364